In [25]:
import gzip
import ujson
import pandas as pd
import sklearn as sk
import numpy as np

In [7]:
filedict = []
with gzip.open("data/yelp_train_academic_dataset_business.json.gz",'rb') as f:
    for line in f:
        tempdict= ujson.loads(line)
        filedict.append(tempdict)

In [11]:
df = pd.DataFrame(filedict)

## City_model

In [20]:
city_df = df[['city','stars']]

In [32]:
gcity = city_df.groupby("city").agg({'stars':np.mean}).reset_index().head()

In [33]:
gcity.head()

,city,stars
0,Ahwatukee,3.687500
1,Anthem,3.781818
2,Apache Junction,3.637500
3,Arcadia,5.000000
4,Atlanta,3.500000


In [97]:
gcity[gcity.city=="Atlanta"].stars.values[0]

3.5

In [131]:
class City_model():
    def __init__(self):
        pass
    def fit(self, city_df):
        self.mean = city_df.stars.mean()
        self.city_grouped = city_df.groupby('city').agg({'stars':np.mean}).reset_index()
    def predict(self, name):
        if name not in self.city_grouped.city.unique():
            return self.mean
        else:
            return self.city_grouped[self.city_grouped.city==name].stars.values[0]

In [132]:
city_model = City_model()

In [133]:
city_model.fit(city_df)

In [134]:
city_model.predict("Atlanta")

3.5

In [135]:
import dill

In [136]:
with open('city_model.pkl', 'wb') as f:
    dill.dump(city_model,f)

In [137]:
with open('city_model.pkl', 'rb') as f:
    model = dill.load(f)

In [139]:
model.predict("Atanta")

3.6729137013021247

## lat_long_model

In [140]:
df.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,{u'By Appointment Only': True},vcNAWiLM4dR7D2nwwJ7nCA,"[Doctors, Health & Medical]",Phoenix,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","{u'Thursday': {u'close': u'17:00', u'open': u'...",33.499313,-111.983758,"Eric Goldberg, MD",[],True,7,3.5,AZ,business
1,"{u'Take-out': True, u'Price Range': 1, u'Outdo...",JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants],De Forest,"6162 US Highway 51\nDe Forest, WI 53532",{},43.238893,-89.335844,Pine Cone Restaurant,[],True,26,4.0,WI,business
2,"{u'Take-out': True, u'Outdoor Seating': False,...",uGykseHzyS5xAMWoN6YUqA,"[American (Traditional), Restaurants]",De Forest,"505 W North St\nDe Forest, WI 53532","{u'Monday': {u'close': u'22:00', u'open': u'06...",43.252267,-89.353437,Deforest Family Restaurant,[],True,16,4.0,WI,business
3,"{u'Take-out': True, u'Accepts Credit Cards': T...",LRKJF43s9-3jG9Lgx4zODg,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R...",De Forest,"4910 County Rd V\nDe Forest, WI 53532","{u'Monday': {u'close': u'22:00', u'open': u'10...",43.251045,-89.374983,Culver's,[],True,7,4.5,WI,business
4,"{u'Take-out': True, u'Has TV': False, u'Outdoo...",RgDg-k9S5YD_BaxMckifkg,"[Chinese, Restaurants]",De Forest,"631 S Main St\nDe Forest, WI 53532","{u'Monday': {u'close': u'22:00', u'open': u'11...",43.240875,-89.343722,Chang Jiang Chinese Kitchen,[],True,3,4.0,WI,business


In [147]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import grid_search

In [142]:
rf = RandomForestRegressor()

In [149]:
gs = grid_search.GridSearchCV(
    rf,
    {"max_depth":range(1,11)},
    cv = 5,
    n_jobs=2,
    scoring="mean_squared_error"
)

In [143]:
latlongdf = df[['latitude','longitude']]

In [150]:
gs.fit(latlongdf, df.stars)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=2,
       param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='mean_squared_error', verbose=0)

In [151]:
print gs.best_params_

{'max_depth': 8}


In [152]:
gs.predict([43.252267,-89.353437])

array([ 3.69042307])

In [153]:
with open('latlong.pkl', 'wb') as f:
    dill.dump(gs,f)